In [29]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
os.chdir('/Users/armitasafavi/downloads')

In [2]:
#creating the json file
with open('world_bank_projects.json', 'r') as json_file:
    json_data = json.load(json_file)

In [15]:
#use normalization to create tables from nested elements
projects = json_normalize(json_data)
project_themes = json_normalize(json_data, 'mjtheme_namecode', errors='ignore')

In [31]:
#exploring the dataset
projects.head(5)

,_id.$oid,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,source,status,supplementprojectflg,theme1.Name,theme1.Percent,theme_namecode,themecode,totalamt,totalcommamt,url
0,52b213b38594d8a2be17c780,1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,IBRD,Active,N,Education for all,100,"[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,52b213b38594d8a2be17c781,2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,IBRD,Active,N,Other economic management,30,"[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,52b213b38594d8a2be17c782,2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,IBRD,Active,Y,Regional integration,46,"[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,52b213b38594d8a2be17c783,2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,IBRD,Active,N,Participation and civic engagement,50,"[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,52b213b38594d8a2be17c784,2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,IBRD,Active,N,Export development and competitiveness,30,"[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [32]:
project_themes.head(5)

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [16]:
#Finding the 10 countries with most projects
country_list =projects['countryshortname'].value_counts()
Top_10 = pd.DataFrame(country_list)
Top_10.head(10)

,countryshortname
Indonesia,19
China,19
Vietnam,17
India,16
"Yemen, Republic of",13
Bangladesh,12
Morocco,12
Nepal,12
Mozambique,11
Africa,11


In [23]:
#Replacing empty strings with NaNs
project_themes = project_themes.replace('',np.nan, regex=True)
#Find the top 10 major project themes
project_themes['name'].value_counts()

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: name, dtype: int64

In [24]:
project_themes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
code    1499 non-null object
name    1377 non-null object
dtypes: object(2)
memory usage: 23.5+ KB


In [26]:
#creating a dict to match unique codes to themes
listOfDicts=projects['mjtheme_namecode']
codes = {}

for i in range(0,len(listOfDicts)):
    for j in range(0, len(listOfDicts[i])):
        if listOfDicts[i][j].get('code') not in codes and listOfDicts[i][j].get('name') != '':
            codes[listOfDicts[i][j].get('code')] = listOfDicts[i][j].get('name')
        



In [35]:
codes

{'8': 'Human development',
 '1': 'Economic management',
 '6': 'Social protection and risk management',
 '5': 'Trade and integration',
 '2': 'Public sector governance',
 '11': 'Environment and natural resources management',
 '7': 'Social dev/gender/inclusion',
 '4': 'Financial and private sector development',
 '10': 'Rural development',
 '9': 'Urban development',
 '3': 'Rule of law'}

In [36]:
#filling NaNs by looking up names to codes
project_themes.name = project_themes.name.fillna(project_themes.code.map(codes))

In [37]:
#check for NaN values
project_themes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 23.5+ KB
